# Lab 5: Adaptive Video Streaming
This lab explores the trade-off between different metrics of video quality (average rate, interruptions, and variability of rate) in an adaptive video delivery system.

**Credits:** This lab is based on [Fraida Fund's Adaptive video experiments](https://witestlab.poly.edu/blog/adaptive-video-reproducing/). Refer to this page for additional information and resources.

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

In [ ]:
fablib = fablib_manager() 
conf = fablib.show_config()

In [ ]:
slice_name="video-streaming-lab" + fablib.get_bastion_username()

In [ ]:
slice = None
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
except:
    print("You don't have a slice named %s yet." % slice_name)

## Setting up plot-generating functions for later
The following cells set up functions that we can call later to generate plots for our data and to download our files from the nodes.

In [ ]:
'''
This function will read in the data and plot the video rate against time (higher rate means higher quality video!)
The background of the plot will be shaded according to the video playback status: a light cyan background means the video is playing, while a light pink background means the video is buffering ("frozen").
'''
def plot_video_rate(filename: str) -> None:
    c = {'INITIAL_BUFFERING': 'violet', 'PLAY': 'lightcyan', 'BUFFERING': 'lightpink'}

    dash = pd.read_csv(filename)
    dash = dash.loc[dash.CurrentPlaybackState.isin(c.keys() )]
    states = pd.DataFrame({'startState': dash.CurrentPlaybackState[0:-2].values, 'startTime': dash.EpochTime[0:-2].values,
                            'endState':  dash.CurrentPlaybackState[1:-1].values, 'endTime':   dash.EpochTime[1:-1].values})
    
    
    for index, s in states.iterrows():
      plt.axvspan(s['startTime'], s['endTime'],  color=c[s['startState']], alpha=1)
    
    plt.plot(dash[dash.Action!="Writing"].EpochTime, dash[dash.Action!="Writing"].Bitrate, 'kx:')
    plt.title("Video rate (bps)");
    plt.xlabel("Time (s)");

In [ ]:
'''
We can also visualize the buffer occupancy over time. This function does that.
In the generated plot, the line shows the number of segments in the buffer over time, and the colored background indicates whether the client is playing video (light cyan) or buffering (light pink). 
When the buffer occupancy goes to zero, the will have to stop playing in order to retrieve more data into the buffer.
'''
def plot_buffer(filename: str) -> None:
    c = {'INITIAL_BUFFERING': 'violet', 'PLAY': 'lightcyan', 'BUFFERING': 'lightpink'}

    dash = pd.read_csv(filename)
    dash = dash.loc[dash.CurrentPlaybackState.isin(c.keys() )]
    states = pd.DataFrame({'startState': dash.CurrentPlaybackState[0:-2].values, 'startTime': dash.EpochTime[0:-2].values,
                            'endState':  dash.CurrentPlaybackState[1:-1].values, 'endTime':   dash.EpochTime[1:-1].values})
    
    
    for index, s in states.iterrows():
      plt.axvspan(s['startTime'], s['endTime'],  color=c[s['startState']], alpha=1)
    
    plt.plot(dash[dash.Action!="Writing"].EpochTime, dash[dash.Action!="Writing"].CurrentBufferSize, 'kx:')
    plt.title("Buffer(segments)");
    plt.xlabel("Time (s)");

In [ ]:
def download_csv(f: str) -> str:
    ASTREAM_LOGS = "ASTREAM_LOGS/" # do not change this (unless you rename the directory)
    node_filepath = ASTREAM_LOGS + f
    local_filepath = os.path.join(os.getcwd() + '/' + f)
    filename = local_filepath
    slice.get_node("romeo").download_file(local_filepath, node_filepath)
    return filename

In [ ]:
def download_mp4(client_video_save_dir: str) -> None:
    mp4_name = '/BigBuckBunny.mp4'
    node_filepath = client_video_save_dir + mp4_name
    local_filepath = os.path.join(os.getcwd() + mp4_name)
    slice.get_node("romeo").download_file(local_filepath, node_filepath)

## Experiment 1: constant bit rate

On the router, set a constant bit rate of 1000 Kbits/second with
```
bash rate-set.sh 1000Kbit
```
(The first time you run it, you may see an error referencing a problem deleting a qdisc, but you can safely ignore this error.)

Note: you can specify a data rate in Kbits/second using `Kbit` or in Mbits/second using `Mbit`.

Then, on the client ("romeo"), start the DASH player with the "basic" adaptation policy:
```
python3 ~/AStream/dist/client/dash_client.py -m http://juliet/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```

(Optional: you can alternatively try `netflix` or `sara` as the DASH policy.)

Leave this running for a while. Then, you can interrupt the DASH client with Ctrl+C.

To understand the performance of the DASH policy, we can look at the logs produced by the client. These will be located inside a directory named `ASTREAM_LOGS` in your home directory on the "client" node. Use
```
ls ~/ASTREAM_LOGS
```
to find these.

Use the following cell to retrieve the log with the .csv extension. Set the variable `csv_file` to the correct filename:

In [ ]:
csv_file_ex1 = 'DASH_BUFFER_LOG_2025-10-22.12_35_13.csv' # update this variable

In [ ]:
filename = download_csv(csv_file_ex1)

The last following cell will read in the data and plot the video rate against time (higher rate means higher quality video!)

The background of the plot will be shaded according to the video playback status: a light cyan background means the video is playing, while a light pink background means the video is buffering ("frozen").

In [ ]:
plot_video_rate(filename)

We can also visualize the buffer occupancy over time. In the following plot, the line shows the number of segments in the buffer over time, and the colored background indicates whether the client is playing video (light cyan) or buffering (light pink). When the buffer occupancy goes to zero, the will have to stop playing in order to retrieve more data into the buffer.

In [ ]:
plot_buffer(filename)

To re-create the video, run the following inside the directory where the client saved the video. The path looks something like "~/TEMP_<random_str>".
```
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy BigBuckBunny.mp4
```
Then, you can use the following cells to download the streamed video. Once it is downloaded, you can download the file from JupyterHub to your laptop and play it on VLC media player. In the following cell, set the correct directory where the files are:

In [ ]:
client_video_save_dir = 'TEMP_JJZwkD' # update this variable with your directory's name

In [ ]:
download_mp4(client_video_save_dir)

**In this cell, discuss what you observed with how the file was streamed. You can discuss the plots you generated and the video file you downloaded. You may talk about how the video looked like, what the plots represent, whether or not the plots correspond to what you see in the video itself. Write your answer below:**



## Experiment 2: constant bit rate with interruption
In the experiment with constant bit rate, you may not have experienced any rebuffering.

To see how the video client works when there is a temporary interruption in the network, try repeating this experiment, but during the video session, reduce the network data rate to a very low value in middle of the session.

On the "router", set a constant bit rate of 1000 Kbits/second with
```
bash rate-set.sh 1000Kbit
```
Then, on the client ("romeo"), start the DASH player with the "basic" adaptation policy:
```
python3 ~/AStream/dist/client/dash_client.py -m http://juliet/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```
Leave this running for one minute (60 seconds). Then, on the "router", reduce the network data rate to 100 Kbits/second:
```
bash rate-set.sh 100Kbit
```
After another 60 seconds, restore the original data rate:
```
bash rate-set.sh 1000Kbit
```
After a little while longer, stop the video client on "romeo" with Ctrl+C.

Repeat the data analysis by plotting figures for your data using the cells below. This time, you will note that when the network rate decreases, the buffer occupancy starts to drop, until it reaches zero and the client has to pause for rebuffering. When it is able to retrieve another segment, it will play it back, but if it has overestimated the download rate, then it will have to pause for rebuffering again. Eventually, the network data rate increases again, and then the video resumes playback at a higher rate and without interruptions.

In [ ]:
csv_file_ex2 = 'DASH_BUFFER_LOG_2025-10-22.12_49_04.csv' # update this variable

In [ ]:
filename = download_csv(csv_file_ex2)

In [ ]:
plot_video_rate(filename)

In [ ]:
plot_buffer(filename)

Same as before. You should re-create the video. To do that, run the following inside the directory where the client saved the video. The path looks something like "~/TEMP_<random_str>".
```
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy BigBuckBunny.mp4
```
Then, you can use the following cells to download the streamed video. Once it is downloaded, you can download the file from JupyterHub to your laptop and play it on VLC media player. In the following cell, set the correct directory where the files are:

In [ ]:
client_video_save_dir = 'TEMP_zVvz5X' # update this variable with your directory's name

In [ ]:
download_mp4(client_video_save_dir)

**In this cell, discuss what you observed with how the file was streamed. You can discuss the plots you generated and the video file you downloaded. You may talk about how the video looked like, what the plots represent, whether or not the plots correspond to what you see in the video itself. Did you notice any buffering in the video? Write your answer below:**


## Experiment 3: compare adaptive video policies
As in the previous experiment, in this experiment we will use a constant bit rate, with a brief interruption. However, we will compare the way two video adaptation policies react to this interruption - we'll compare a *rate based policy* (`basic`) and a *buffer based policy* (`netflix`) under identical settings (similar "high" network rate, "low" data rate, similar duration of the "high" data rate before the interruption, and and similar duration of the "interruption").

### Rate-based vs. buffer-based policies
The basic rate adaptation policy (`basic`) chooses a video rate based on the observed download rate. It keeps track of the average download time for recent segments, and calculates a running average.

If the download rate exceeds the current video rate by some threshold, it may increase the video rate. If the download rate is lower than the current video rate, it decreases the video rate to ensure smooth playback.

You can see the source code for the `basic` policy here: [basic_dash2.py](https://github.com/teaching-on-testbeds/AStream/tree/master/dist/client/adaptation/basic_dash2.py)

The buffer-based policy (`netflix`) adapts the video rate based on the current buffer occupancy, rather than the current download rate. When there are many segments already buffered, it can increase the video rate; if the buffer occupancy is low and the client is at risk of rebuffering, it must decrease the video rate.

You can see the source code for the `netflix` policy here: [netflix_dash.py](https://github.com/teaching-on-testbeds/AStream/tree/master/dist/client/adaptation/netflix_dash.py). This policy is based on the paper:
>Te-Yuan Huang, Ramesh Johari, and Nick McKeown. 2013. Downton abbey without the hiccups: buffer-based rate adaptation for HTTP video streaming. In Proceedings of the 2013 ACM SIGCOMM workshop on Future human-centric multimedia networking (FhMN '13). Association for Computing Machinery, New York, NY, USA, 9–14. https://doi.org/10.1145/2491172.2491179

The policy defines two buffer occupancy thresholds: reservoir (defaults to 10%) and cushion (defaults to 90%). If the buffer occupancy is below the reservoir threshold, it selects the minimum video rate to fill the buffer quickly. If the buffer is within the reservoir and cushion range, it selects a video rate using a rate map function that maps buffer occupancy to video rate according to some increasing function. If the buffer occupancy exceeds the cushion threshold, it selects the maximum video rate.

The behavior of each policy is parameterized by the settings defined in the code base at: [config_dash.py](https://github.com/teaching-on-testbeds/AStream/tree/master/dist/client/config_dash.py).

Key factors defined here for the rate based ("basic") policy include:

- `BASIC_THRESHOLD` - the maximum number of segments to store in the buffer in the rate based ("basic") policy.
- `BASIC_UPPER_THRESHOLD` - to avoid oscillations, in the rate based ("basic") policy, the video rate increases or decreases only if it is different from the download rate by at least this factor.
- `BASIC_DELTA_COUNT` - the number of segments' download rate to include in the moving average of network download rate. The smaller this number, the more reactive the client is to (potentially transient!) changes in network rate.

Key factors defined here for the buffer based ("netflix") policy include:

- `NETFLIX_RESERVOIR` - the value of the "reservoir" described above, as a fraction of total buffer size.
- `NETFLIX_CUSHION` - the value of the "cushion" described above, as a fraction of total buffer size.
- `NETFLIX_BUFFER_SIZE` - the maximum number of segments to store in the buffer in the buffer based ("netflix") policy.
- `NETFLIX_INITIAL_BUFFER` and `NETFLIX_INITIAL_FACTOR` - these define the behavior of the policy in the initial stage, which is a bit different than the approach described above.

### Ex 3a: Experiment for the rate based policy
On the "router", we will configure the following "sequence" of network rates -

- 1000 Kbits/second for 100 seconds
- 350 Kbits/second for another 75 seconds
- 2000 Kbits/second for another 125 seconds
and while this is ongoing, we will run a DASH client with the rate based policy. After 300 seconds, we will stop the DASH client.

To realize this sequence of network rates, on the router, run

```
bash rate-set.sh 1000Kbit; echo "Start the DASH client"; sleep 100; bash rate-set.sh 350Kbit; sleep 75; bash rate-set.sh 2000Kbit; sleep 125; echo "Stop the DASH client"
```
On the client ("romeo"), immediately start the DASH player with the "basic" adaptation policy:

```
python3 ~/AStream/dist/client/dash_client.py -m http://juliet/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```

Let the DASH player run for 300 seconds, then stop the video client on "romeo" with Ctrl+C.

Repeat the data analysis steps as before by running the following cells. Save the figures and the reconstructed video for the rate based policy, so that after completing the experiment in the next section with the buffer based policy, you can compare their behavior.

In [ ]:
csv_file_ex3a = 'DASH_BUFFER_LOG_2025-10-22.13_01_21.csv' # update this variable

In [ ]:
filename = download_csv(csv_file_ex3a)

In [ ]:
plot_video_rate(filename)

In [ ]:
plot_buffer(filename)

Same as before. You can re-create the video. To do that, run the following inside the directory where the client saved the video. The path looks something like "~/TEMP_<random_str>".
```
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy BigBuckBunny.mp4
```
Then, you can use the following cells to download the streamed video. Once it is downloaded, you can download the file from JupyterHub to your laptop and play it on VLC media player. In the following cell, set the correct directory where the files are:

In [ ]:
client_video_save_dir = 'TEMP_zVvz5X' # update this variable with your directory's name

In [ ]:
download_mp4(client_video_save_dir)

**In this cell, discuss what you observed with how the file was streamed. You can discuss the plots you generated and the video file you downloaded. You may talk about how the video looked like, what the plots represent, whether or not the plots correspond to what you see in the video itself. How is it different from the videos you observed before? Write your answer below:**


### Ex 3b: Experiment for the buffer based policy
We will now repeat the same experiment for the buffer based policy.

On the router, run

```
bash rate-set.sh 1000Kbit echo "Start the DASH client" sleep 100 bash rate-set.sh 350Kbit sleep 75 bash rate-set.sh 2000Kbit sleep 125 echo "Stop the DASH client"
```

Then, on the client ("romeo"), start the DASH player with the "netflix" adaptation policy:

```
python3 ~/AStream/dist/client/dash_client.py -m http://juliet/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'netflix' -d
```

Let the DASH player run for 300 seconds, then stop the video client on "romeo" with Ctrl+C.

Use the following cells to repeat the data analysis steps as before. Save the figures and the reconstructed video for the buffer based policy, for comparison with the rate based policy in the previous section.

In [ ]:
csv_file_ex3b = 'DASH_BUFFER_LOG_2025-10-22.13_01_21.csv' # update this variable

In [ ]:
filename = download_csv(csv_file_ex3b)

In [ ]:
plot_video_rate(filename)

In [ ]:
plot_buffer(filename)

Same as before. You can re-create the video. To do that, run the following inside the directory where the client saved the video. The path looks something like "~/TEMP_<random_str>".
```
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy BigBuckBunny.mp4
```
Then, you can use the following cells to download the streamed video. Once it is downloaded, you can download the file from JupyterHub to your laptop and play it on VLC media player. In the following cell, set the correct directory where the files are:

In [ ]:
client_video_save_dir = 'TEMP_zVvz5X' # update this variable with your directory's name

In [ ]:
download_mp4(client_video_save_dir)

**In this cell, discuss what you observed with how the file was streamed. You can discuss the plots you generated and the video file you downloaded. You may talk about how the video looked like, what the plots represent, whether or not the plots correspond to what you see in the video itself. How is it different from the video you observed in Ex 3a and others? Write your answer below:**


## (Optional) Experiment 4: mobile user
This is an optional experiment. You can try to experience adaptive video as a mobile user! Repeat the experiment, but instead of setting a constant data rate on the router, you can let it play back a "trace" file with e.g.

```
bash rate-vary.sh ~/Dataset_1/Dataset/Ferry/Ferry5.csv 0.1
```

where the first argument is the path to a trace file, and the second argument is a scaling factor greater than 0 but less than 1. (The smaller the scaling factor, the lower the network quality while still preserving the trace dynamics.)

Then, on the client, run

```
python3 ~/AStream/dist/client/dash_client.py -m http://juliet/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```

wait a minute or two, then stop the video player with Ctrl+C. Repeat the data analysis (you may copy re-use/copy the cells/function from above for this).

The following figure shows the "dynamics" (throughput in Mbps against time) for each of the traces:
![](./images/nyc-traces.png)

In [ ]:
# You may copy the cells/function/code from above and do the analysis here.

**Here, you may write down your observations regarding how this experiment compares with the previous ones. You may discuss the video playback quality, the data in the plots, you own opinions and thoughts, etc.**